In [ ]:
#creating a methode for the UFL to get text from the file 
from nbformat import reader
import PyPDF2

class UnstructuredFileLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def get_text(self):
        pdf_file = open(self.file_path, 'rb')
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        text = ""
        #going through each page of the pdf
        for page_num in range(len(reader.pages)):
            page = pdf_reader.getPage(page_num)
            #extract the page content & store it in the text var
            text += page.extract_text()
        pdf_file.close()
        return text


In [40]:
import tempfile
from langchain_community.document_loaders import UnstructuredFileLoader
from nltk.tokenize import sent_tokenize
import PyPDF2

def extract_text_from_pdf(pdf_path):
    # Use PyPDF2 to extract text from the PDF
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ''
    # Loop through each page of the PDF
    for page_number in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_number]
        text += page.extract_text()
    pdf_file.close()
    return text



def chunk_text(text):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < 500:  
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def save_chunks_to_file(chunks, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for chunk in chunks:
            f.write(chunk + '\n')

pdf_path = 'infos/pdf2.pdf'
pdf_text = extract_text_from_pdf(pdf_path)

loader = UnstructuredFileLoader(pdf_text)

processed_text = loader

# Print the processed text
print(processed_text)